In [ ]:
# Tuddal coordinates: 59.7463 N  8.8083 E

# Constants

epsilon_surface = 0.97  # Emissivity of the surface
sigma = 5.67e-8  # Stefan-Boltzmann constant (W/m^2 K^4)

lambda_vaporization = 2.5e6  # Latent heat of vaporization (J/kg)
rho_air = 1.225  # Air density (kg/m^3)
C_e = 0.0015  # Bulk transfer coefficient for latent heat

Cp = 1005  # Specific heat capacity of air at constant pressure (J/(kg*K))
Ch = 0.001  # Bulk transfer coefficient for sensible heat
#––––––––––––––––––

alpha_snow = 0.8  # Albedo for snow-covered surface
alpha_soil_water = 0.15  # Albedo for soil or water surface
alpha_cloud = 2  # Example parameter for cloud fraction Beta distribution
beta_cloud = 5  # Example parameter for cloud fraction Beta distribution


In [1]:
import numpy as np

# Function to calculate saturation specific humidity
def saturation_specific_humidity(T):
    """Calculate saturation specific humidity (in kg/kg) at temperature T (Celsius)."""
    e_s = 6.112 * np.exp((17.67 * T) / (T + 243.5))  # Saturation vapor pressure in hPa
    q_sat = 0.622 * (e_s / (1013.25 - e_s))  # Specific humidity formula
    return q_sat

# Function to calculate specific humidity at the surface (q_s)
def surface_specific_humidity(T_s, snow_cover):
    """Calculate specific humidity at the surface based on temperature and snow cover."""
    q_sat_surface = saturation_specific_humidity(T_s)
    if snow_cover == 1:  # Snow-covered
        q_s = q_sat_surface * alpha_snow
    else:  # Snow-free (soil/water)
        q_s = q_sat_surface * alpha_soil_water
    return q_s

# Function for outgoing shortwave radiation
def outgoing_shortwave_radiation(SW_in, snow_cover):
    albedo = alpha_snow if snow_cover == 1 else alpha_soil_water
    SW_out = albedo * SW_in
    return SW_out

# Function for outgoing longwave radiation
def outgoing_longwave_radiation(T_s):
    LW_out = epsilon_surface * sigma * (T_s + 273.15)**4
    return LW_out

# Function for sensible heat flux
def sensible_heat_flux(T_s, T_a, u):
    SH = rho_air * Cp * Ch * u * (T_s - T_a)
    return SH

# Function for latent heat flux
def latent_heat_flux(q_s, q_a, u):
    LE = lambda_vaporization * rho_air * C_e * u * (q_s - q_a)
    return LE

# Stochastic cloud fraction and LW_in calculation
def cloud_fraction_and_LW_in(T_a):
    CF = np.random.beta(alpha_cloud, beta_cloud)
    LW_clear = sigma * (T_a + 273.15)**4
    LW_cloudy = 1.2 * LW_clear
    LW_in = (1 - CF) * LW_clear + CF * LW_cloudy
    return CF, LW_in

# Surface temperature update based on energy balance
def update_surface_temperature(T_s, SW_in, LW_in, LE, SH, snow_cover, C_s, time_step=3600):
    SW_out = outgoing_shortwave_radiation(SW_in, snow_cover)
    LW_out = outgoing_longwave_radiation(T_s)
    net_radiation = SW_in - SW_out + LW_in - LW_out - LE - SH
    dT_s = (net_radiation / C_s) * time_step
    T_s_updated = T_s + dT_s
    return T_s_updated

Outgoing Shortwave Radiation (SW_out): 160.0
Outgoing Longwave Radiation (LW_out): 284.3583583822426
Sensible Heat Flux (SH): -12.311250000000001
Cloud Fraction (CF): 0.47769821806997037
Incoming Longwave Radiation (LW_in): 330.849927735018
Latent Heat Flux (LE): 1.8612790657335032
Updated Surface Temperature (T_s): 343.98954503335085


In [ ]:
# Example scenario variables
T_s_example = -5  # Initial surface temperature in Celsius
T_a_example = -3  # Air temperature in Celsius
SW_in_example = 200  # Example incoming shortwave radiation in W/m^2
u_example = 5  # Wind speed in m/s
q_a_example = 0.002  # Specific humidity in the air (example)
C_s_example = 1000  # Surface heat capacity in J/(m^2*K)
snow_cover_example = 1  # Snow presence

# Calculations
SW_out_example = outgoing_shortwave_radiation(SW_in_example, snow_cover_example)
LW_out_example = outgoing_longwave_radiation(T_s_example)
SH_example = sensible_heat_flux(T_s_example, T_a_example, u_example)
CF_example, LW_in_example = cloud_fraction_and_LW_in(T_a_example)
LE_example = latent_heat_flux(surface_specific_humidity(T_s_example, snow_cover_example), q_a_example, u_example)
T_s_updated_example = update_surface_temperature(T_s_example, SW_in_example, LW_in_example, LE_example, SH_example, snow_cover_example, C_s_example)

# Display results
print("Outgoing Shortwave Radiation (SW_out):", SW_out_example)
print("Outgoing Longwave Radiation (LW_out):", LW_out_example)
print("Sensible Heat Flux (SH):", SH_example)
print("Cloud Fraction (CF):", CF_example)
print("Incoming Longwave Radiation (LW_in):", LW_in_example)
print("Latent Heat Flux (LE):", LE_example)
print("Updated Surface Temperature (T_s):", T_s_updated_example)
